In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/train.csv')

In [4]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.models import Sequential

In [6]:
import nltk
from nltk.corpus import stopwords
import re

In [8]:
df.dropna(inplace=True)

In [9]:
df.reset_index(inplace=True)
X = df.drop('label',axis = 1)
Y = df['label']

In [13]:
X.shape

(18285, 3)

In [12]:
X.drop(['index','id'],axis = 1,inplace=True)

In [14]:
from nltk.stem.porter import PorterStemmer

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
ps = PorterStemmer()
corpus= []
for i in range(0,len(X)):
  title = re.sub('^[a-zA-Z]',' ',X['title'][i])
  title = title.lower()
  title = title.split()
  title = [ps.stem(word) for word in title if not word in stopwords.words('english')]
  title = ' '.join(title)
  corpus.append(title)

In [21]:
voc_size = 5000
one_hot_repr = [one_hot(words,voc_size) for words in corpus]

In [23]:
max_size = 20
embedded_repr = pad_sequences(one_hot_repr,padding='pre',maxlen=max_size)

In [24]:
embedded_repr

array([[   0,    0,    0, ..., 2210, 2794, 4474],
       [   0,    0,    0, ..., 3495, 3067, 1144],
       [   0,    0,    0, ..., 4395, 4389, 2277],
       ...,
       [   0,    0,    0, ...,  626, 3607, 4113],
       [   0,    0,    0, ..., 3754,  252, 2609],
       [   0,    0,    0, ..., 1305, 2453, 3917]], dtype=int32)

In [32]:
embedding_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length=max_size))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [30]:
import numpy as np
X_final = np.array(embedded_repr)

In [33]:
model.fit(X_final,Y,validation_split=0.2,epochs=10,batch_size=72)

Epoch 1/10
204/204 [==============================] - 9s 9ms/step - loss: 0.3298 - accuracy: 0.8398 - val_loss: 0.1938 - val_accuracy: 0.9163
Epoch 2/10
204/204 [==============================] - 1s 7ms/step - loss: 0.1322 - accuracy: 0.9491 - val_loss: 0.1956 - val_accuracy: 0.9185
Epoch 3/10
204/204 [==============================] - 1s 7ms/step - loss: 0.0856 - accuracy: 0.9692 - val_loss: 0.2324 - val_accuracy: 0.9212
Epoch 4/10
204/204 [==============================] - 1s 7ms/step - loss: 0.0534 - accuracy: 0.9832 - val_loss: 0.2897 - val_accuracy: 0.9166
Epoch 5/10
204/204 [==============================] - 1s 7ms/step - loss: 0.0298 - accuracy: 0.9908 - val_loss: 0.3084 - val_accuracy: 0.9136
Epoch 6/10
204/204 [==============================] - 1s 7ms/step - loss: 0.0165 - accuracy: 0.9951 - val_loss: 0.3927 - val_accuracy: 0.9139
Epoch 7/10
204/204 [==============================] - 1s 7ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 0.4120 - val_accuracy: 0.9027
Epoch 

In [35]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test  =train_test_split(X_final,Y,test_size = 0.25)

In [36]:
x_test,y_test

(array([[   0,    0,    0, ...,  465,  365, 3345],
        [   0,    0,    0, ...,  626, 3607, 4113],
        [   0,    0,    0, ...,  626, 3607, 4113],
        ...,
        [   0,    0,    0, ..., 1144,  341, 1144],
        [   0,    0,    0, ..., 3108, 4498,   82],
        [   0,    0,    0, ..., 3929, 1921,  167]], dtype=int32), 7892     1
 3821     0
 3106     0
 18270    1
 14601    1
         ..
 17506    1
 1605     1
 10749    0
 3854     1
 6069     1
 Name: label, Length: 4572, dtype: int64)

In [38]:
pred = model.predict(x_test)

In [39]:
model.evaluate(x_test, y_test, verbose=2)

143/143 - 0s - loss: 0.1126 - accuracy: 0.9834 - 351ms/epoch - 2ms/step


[0.11262740194797516, 0.9833770990371704]